<a href="https://colab.research.google.com/github/ThanhLOL123/WEB/blob/master/Fine-Tuning%20BART%20for%20Text%20Summarization%20Using%20Parameter-Efficient%20Fine-Tuning%20(PEFT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [3]:
from datasets import load_dataset
# Đây là dataset chứa thông tin tóm tắt các tin tức
dataset = load_dataset("vietgpt/news_summarization_vi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/65361 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 65361
    })
    test: Dataset({
        features: ['content', 'summary'],
        num_rows: 1000
    })
})


In [5]:
total_rows = len(dataset["train"])

# Do giới hạn về năng lực tính toán của colab free. Trong đó nếu mà fine-tuned peft có thể mất đến hơn 17h với năng lực phần cứng mà tôi có. Nên tôi đã quyết định giảm số lượng dataset
num_rows_to_keep = total_rows - 58000
if num_rows_to_keep < 0:
    raise ValueError("Cannot remove more rows than the dataset size.")


reduced_train_dataset = dataset["train"].select(range(num_rows_to_keep))
reduced_test_dataset = dataset["train"].select(range(1500))

reduced_dataset = dataset.copy()
reduced_dataset["train"] = reduced_train_dataset
reduced_dataset["test"] = reduced_test_dataset
print(f"Reduced dataset size: {len(reduced_dataset['train'])}")
print(f"Reduced dataset size: {len(reduced_dataset['test'])}")

Reduced dataset size: 7361
Reduced dataset size: 1500


In [6]:
from datasets import Dataset, DatasetDict

# Extract individual splits as dictionaries
train_dataset = reduced_dataset['train']
test_dataset = reduced_dataset['test']

# Combine into a DatasetDict
rdataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,

})

print(rdataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 7361
    })
    test: Dataset({
        features: ['content', 'summary'],
        num_rows: 1500
    })
})


In [7]:
# Tôi chọn bart. Bởi vì nó phù hợp cho bài toán tóm tắt văn bản nhờ vào kiến trúc encoder-decoder, khả năng fine-tune trên các tập dữ liệu lớn và xử lý hiệu quả các biến thể của văn bản.
#Mô hình này kết hợp sự hiểu biết ngữ cảnh của BERT và khả năng tạo văn bản tuần tự của GPT, cho phép tạo ra các bản tóm tắt chính xác và tự nhiên.

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [10]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 406291456
all model parameters: 406291456
percentage of trainable model parameters: 100.00%


In [11]:
index = 200
original_model_summaries = []
dialogue = dataset['test'][index]['content']
summary = dataset['test'][index]['summary']

prompt = f"""
Tóm tắt văn bản dưới đây.

{dialogue}

Tóm tắt:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=400,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
original_model_summaries.append(output)

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Tóm tắt văn bản dưới đây.

Theo đó, các Sở GD&ĐT chỉ đạo các trường phổ thông thực hiện nghiêm túc việc may, mặc đồng phục trong nhà trường phải được sự đồng ý của hội đồng trường và thống nhất với ban đại diện cha mẹ HS. Không bắt buộc HS phải mặc đồng phục hằng ngày đến trường. Ở những nơi có điều kiện và ban đại diện cha mẹ HS đề nghị thì chỉ cần quy định HS mặc đồng phục một buổi/tuần. Nhà trường tạo điều kiện thuận lợi, cho phép gia đình HS tự mua sắm đồng phục HS theo mẫu của trường.
Đối với các trường ĐH, CĐ và TCCN, không bắt buộc HSSV phải mặc đồng phục khi đến trường. Nhà trường ban hành quy định cụ thể về trang phục của HSSV khi đến trường, đảm bảo sự nghiêm túc của môi trường giáo dục nhưng không gò bó, gây khó khăn cho HSSV. Ngoài ra, Bộ còn yêu cầu các Sở GD&ĐT phải rà soát, báo cáo gấp kết quả giám sát và kiểm tra việc thực hiện may, mặc đồng phục của các tr

In [15]:
index = 5
dialogue = dataset['test'][index]['content']
summary = dataset['test'][index]['summary']

prompt = f"""
Tóm tắt văn bản dưới đây.

{dialogue}

Tóm tắt:
"""

inputs = tokenizer(prompt, return_tensors='pt', truncation=True)
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=400,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
original_model_summaries.append(output)

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Tóm tắt văn bản dưới đây.

Ảnh minh họa. (Nguồn: Planespotter.net).
Theo ông Thanh, dự kiến việc kiểm soát tín hiệu chuyển giao cho FIR TP HCM lúc rạng sáng nay nhưng tàu bay chưa liên lạc với FIR của TP HCM. Trước khi vào vùng kiểm soát của ACC TP HCM, máy bay đã mất toàn bộ liên lạc, tín hiệu radar.
Từ thời điểm cơ quan không lưu phía bạn thông báo chuyển giao cho Việt Nam, Trung tâm quản lý bay đường dài (ACC) Hồ Chí Minh đã không nhận được thông tin về chuyến bay. ACC Hồ Chí Minh cũng chưa nhận quyền kiểm soát đã thông báo ngay cho phía bạn về tình trạng này, ông Thanh nói.
Cụ thể, hệ thống radar hàng không không bắt được tín hiệu của chuyến bay. Giờ hiệp đồng chuyển giao giữa hai bên là 16 giờ 50 và giờ dự kiến nhận quyền kiểm soát điều hành bay của Việt Nam là 17 giờ 22 phút, giờ UTC tức 0 giờ 22 phút giờ Việt Nam.
Nhưng ngay trước thời điểm nhận quyền chuyển giao th

In [16]:
index = 10
dialogue = dataset['test'][index]['content']
summary = dataset['test'][index]['summary']

prompt = f"""
Tóm tắt văn bản dưới đây.

{dialogue}

Tóm tắt:
"""

inputs = tokenizer(prompt, return_tensors='pt', truncation=True)
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=400,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
original_model_summaries.append(output)

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Tóm tắt văn bản dưới đây.

Kể từ khi ra mắt MV Hello, Mr. Đàm đã thu hút nhiều sự chú ý khi mạnh tay chi gần 1,5 tỷ đồng cho những trang phục hàng hiệu đắt tiền, mời dàn sao nổi tiếng góp mặt trong MV cùng êkip quay lên đến gần 100 người.
Không chỉ gây sốc về khoản chịu chi, ông hoàng nhạc Việt cũng không ngại lấn sân nhạc trẻ, vừa hát vừa nhảy không thua kém các nghệ sĩ đàn em. Hiện, chỉ sau vài ngày ra mắt, sản phẩm của Đàm Vĩnh Hưng đã thu hút hơn 5 triệu lượt xem và lọt Top 7 thịnh hành trên YouTube.
Mr. Đàm gây chú ý với MV trị giá tiền tỉ.
Tuy vậy, hình ảnh của Mr. Đàm trong ca khúc mới vẫn vấp phải ý kiến trái chiều từ khán giả. Mới đây, một nhóm bạn trẻ còn làm cả video clip reaction (một dạng clip quay lại phản ứng của người xem) cho MV này và liên tục chê bai ông hoàng nhạc Việt.
Ảnh trích từ clip ghi lại phản ứng của nhóm bạn trẻ khi xem MV "Hello" của Mr. Đàm.


In [10]:
#Tiến hành tokenize toàn bộ input
def tokenize_function(example):
    start_prompt = 'Tóm tắt văn bản dưới đây.\n\n'
    end_prompt = '\n\Tóm tắt: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["content"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 2 diff splits: train, test.
tokenized_datasets = rdataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns([ 'content', 'summary',])

Map:   0%|          | 0/7361 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [11]:
from peft import LoraConfig, get_peft_model, TaskType
#PEFT (Parameter-Efficient Fine-Tuning) là phương pháp fine-tune chỉ một phần nhỏ các tham số của mô hình ngôn ngữ lớn như BART, giúp tiết kiệm tài nguyên và giảm nguy cơ overfitting.
#PEFT giữ lại phần lớn các kiến thức đã học trong quá trình pre-training, đảm bảo hiệu suất cao cho các tác vụ NLP.
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # BART
)

In [12]:
peft_model = get_peft_model(original_model,lora_config)

In [13]:
output_dir = f'./peft-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, #
    num_train_epochs=1,
    save_steps=500,

)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [14]:
peft_trainer.train()

peft_model_path="./peft-summary-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.257000
1000,0.971300
1500,0.955800


('./peft-summary-local/tokenizer_config.json',
 './peft-summary-local/special_tokens_map.json',
 './peft-summary-local/vocab.json',
 './peft-summary-local/merges.txt',
 './peft-summary-local/added_tokens.json',
 './peft-summary-local/tokenizer.json')

In [26]:
human_baseline_summaries = []
index = 200
dialogue = dataset['test'][index]['content']
human_baseline_summary = dataset['test'][index]['summary']
human_baseline_summaries.append(human_baseline_summary)
prompt = f"""
Tóm tắt đoạn văn bản.

{dialogue}

Tóm tắt: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Các Sở GD&ĐT đã chỉ đạo các trường phổ thông thực hiện may, mặc đồng phục trong nhà trường với sự đồng ý của hội đồng trường và thống nhất với ban đại diện cha mẹ HS. Không bắt buộc HS phải mặc đồng phục hằng ngày đến trường. Đối với các trường ĐH, CĐ và TCCN, không có bắt buộc mặc đồng phục. Bộ GD&ĐT sẽ rà soát, báo cáo gấp kết quả giám sát và kiểm tra việc thực hiện may, mặc đồng phục của các trường trước ngày 20-9.
---------------------------------------------------------------------------------------------------
PEFT MODEL: Bộ Giáo dục và Đào tạo (GD&ĐT) đã ban hành một đoạn văn bản về việc các trường phổ thông thực hiện nghiêm túc việc may, mặc đồng phục trong nhà trường phải được sự đồng ý của hội đồng trường và thống nhất với ban đại diện cha mẹ HS. Nhà trường tạo điều kiện thuận lợi, cho phép gia đình HS tự mua sắm đồng phục HS theo mẫu của trường. Đối với

In [27]:
peft_model_summaries = []
peft_model_summaries.append(peft_model_text_output)

In [28]:
index = 5
dialogue = dataset['test'][index]['content']
human_baseline_summary = dataset['test'][index]['summary']
human_baseline_summaries.append(human_baseline_summary)
prompt = f"""
Tóm tắt đoạn văn bản.

{dialogue}

Tóm tắt: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')
peft_model_summaries.append(peft_model_text_output)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Máy bay của hãng hàng không Malaysia Airlines chở 239 người trên đường tới Bắc Kinh đã mất liên lạc với đài kiểm soát không lưu sau khi rời thủ đô Kuala Lumpur của Malaysia vào lúc 2 giờ 40 phút ngày 8/3 theo giờ địa phương (18 giờ 40 phút ngày 7/3 theo GMT). Theo ông Thanh, dự kiến việc kiểm soát tín hiệu chuyển giao cho FIR TP HCM lúc rạng sáng nay nhưng tàu bay chưa liên lạc với FIR của TP HCM. Trước khi vào vùng kiểm soát của ACC TP HCM, máy bay đã mất toàn bộ liên lạc, tín hiệu radar. Hiện các đơn vị của Việt Nam đã thống nhất phương án với phía Malaysia, sẵn sàng hỗ trợ khi bạn yêu cầu. Các thông tin mới nhất sẽ được cập nhật và thông báo kịp thời. Máy bay Malaysia Airlines chở 239 người đã mất liên lạc sau khi rời thủ đô Kuala Lumpur vào lúc 2 giờ 40 phút ngày 8/3 theo giờ địa phương (18 giờ 40 phút ngày 7/3 theo GMT). Trước khi vào vùng kiểm soát của ACC T

In [29]:
index = 10
dialogue = dataset['test'][index]['content']
human_baseline_summary = dataset['test'][index]['summary']
human_baseline_summaries.append(human_baseline_summary)
prompt = f"""
Tóm tắt đoạn văn bản.

{dialogue}

Tóm tắt: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')
peft_model_summaries.append(peft_model_text_output)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Mr. Đàm đã thu hút sự chú ý với MV trị giá tiền tỉ và nhảy nhót không thua kém các nghệ sĩ trẻ. Tuy nhiên, hình ảnh của ông đã bị nhóm bạn trẻ chê bai là dị hợm và xa rời công chúng. Mr. Đàm đã lên tiếng phản pháo và cảnh cáo những kẻ chê bai mình là láo lếu, ngông cuồng.
---------------------------------------------------------------------------------------------------
PEFT MODEL: Mr. Đàm đã gây sốc khi mạnh tay chi gần 1,5 tỷ đồng cho những trang phục hàng hiệu đắt tiền, mời dàn sao nổi tiếng góp mặt trong MV cùng êkip quay lên đến gần 100 người. Tuy nhiên, hình ảnh của Mr. Đàm trong ca khúc vẫn vấp phải ý kiến trái chiều từ khán giả. Một nhóm bạn trẻ còn làm cả video clip reaction (một dạng clip quay lại phản ứng của người xem) cho MV này và liên tục chê bai ông hoàng nhạc Việt. Đàm Vĩnh Hưng đã lên tiếng đáp trả và không ngần ngại chỉ trích những kẻ chê bai mì

In [30]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.33731343283582094, 'rouge2': 0.18605757633466644, 'rougeL': 0.271420674405749, 'rougeLsum': 0.271420674405749}
PEFT MODEL:
{'rouge1': 0.6347801767324323, 'rouge2': 0.4972922952174852, 'rougeL': 0.5232911122708082, 'rougeLsum': 0.5232911122708082}




Kết quả bạn đã đạt được cho thấy mô hình BART khi sử dụng phương pháp PEFT (Parameter-Efficient Fine-Tuning) có hiệu suất vượt trội so với mô hình ban đầu:

1. **ROUGE-1**: Tăng từ 0.337 lên 0.635, cho thấy khả năng tóm tắt của mô hình đã cải thiện đáng kể về mức độ chính xác của các từ đơn lẻ.
2. **ROUGE-2**: Tăng từ 0.186 lên 0.497, chứng tỏ mô hình PEFT đã nắm bắt được tốt hơn các cụm từ và cấu trúc ngữ nghĩa quan trọng trong văn bản.
3. **ROUGE-L** và **ROUGE-Lsum**: Tăng từ 0.271 lên 0.523, cho thấy khả năng duy trì các chuỗi từ dài và cấu trúc của văn bản tóm tắt đã được cải thiện rõ rệt.

**Kết luận**: Việc áp dụng PEFT cho mô hình BART đã mang lại sự cải thiện đáng kể về hiệu suất tóm tắt, thể hiện qua các chỉ số ROUGE cao hơn. Trong ví dụ của tôi do hạn chế về phần cứng nên tôi không thể train hết được mô hình. Qua những kết quả trên tuy chưa thể khẳng định được điều gì, nhưng có nói rằng peft có tiềm năng là 1 phương thức giúp tối ưu mô hình trên.

